In [ ]:
!/opt/bin/nvidia-smi

Fri Mar 17 02:22:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    30W /  70W |    105MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/nanjunye/SDNN_python.git

Cloning into 'SDNN_python'...
remote: Enumerating objects: 60902, done.
remote: Counting objects: 100% (59984/59984), done.
remote: Compressing objects: 100% (50564/50564), done.
remote: Total 60902 (delta 9422), reused 59979 (delta 9420), pack-reused 918
Receiving objects: 100% (60902/60902), 76.51 MiB | 28.30 MiB/s, done.
Resolving deltas: 100% (9432/9432), done.
Updating files: 100% (61294/61294), done.


In [ ]:
import sys
sys.path.append('/content/SDNN_python/src')

In [ ]:
from SDNN_cuda import SDNN
from Classifier import Classifier
import numpy as np
import os
from math import floor
import time

In [ ]:
#Settings
start = time.time()
# Flags
learn_SDNN = True  # This flag toggles between Learning STDP and classify features
# or just classify by loading pretrained weights for the face/motor dataset
if learn_SDNN:
    set_weights = False  # Loads the weights from a path (path_set_weigths) and prevents any SDNN learning
    save_weights = True  # Saves the weights in a path (path_save_weigths)
    save_features = True  # Saves the features and labels in the specified path (path_features)
else:
    set_weights = True  # Loads the weights from a path (path_set_weigths) and prevents any SDNN learning
    save_weights = False  # Saves the weights in a path (path_save_weigths)
    save_features = False  # Saves the features and labels in the specified path (path_features)


In [ ]:
# ------------------------------- Learn, Train and Test paths-------------------------------#
ipynb_path = '/content/SDNN_python/'
# Image sets directories
spike_times_learn = [ipynb_path + 'cifar100_test/']
spike_times_train = [ipynb_path + 'cifar100_train/']
spike_times_test = [ipynb_path + 'cifar100_test/']

# Results directories
path_set_weigths = [ipynb_path + 'results/']
path_save_weigths = [ipynb_path + 'results/']
path_features = [ipynb_path + 'results/']

In [ ]:
# ------------------------------- SDNN -------------------------------#
# SDNN_cuda parameters
DoG_params = {'img_size': (32, 32), 'DoG_size': 7, 'std1': 1., 'std2': 2.}  # img_size is (col size, row size)
total_time = 15
network_params = [{'Type': 'input', 'num_filters': 1, 'pad': (0, 0), 'H_layer': DoG_params['img_size'][1],
                       'W_layer': DoG_params['img_size'][0]},
                      {'Type': 'conv', 'num_filters': 4, 'filter_size': 5, 'th': 10.},
                      {'Type': 'pool', 'num_filters': 4, 'filter_size': 7, 'th': 0., 'stride': 6},
                      {'Type': 'conv', 'num_filters': 20, 'filter_size': 17, 'th': 60.},
                      {'Type': 'pool', 'num_filters': 20, 'filter_size': 5, 'th': 0., 'stride': 5},
                      {'Type': 'conv', 'num_filters': 20, 'filter_size': 5, 'th': 2.}]

weight_params = {'mean': 0.8, 'std': 0.01}

max_learn_iter = [0, 1000, 0, 2500, 0, 3000, 0]
stdp_params = {'max_learn_iter': max_learn_iter,
                   'stdp_per_layer': [0, 10, 0, 4, 0, 2],
                   'max_iter': sum(max_learn_iter),
                   'a_minus': np.array([0, .003, 0, .0003, 0, .0003], dtype=np.float32),
                   'a_plus': np.array([0, .004, 0, .0004, 0, .0004], dtype=np.float32),
                   'offset_STDP': [0, floor(network_params[1]['filter_size']),
                                   0,
                                   floor(network_params[3]['filter_size']/8),
                                   0,
                                   floor(network_params[5]['filter_size'])]}

In [ ]:
# Create network
first_net = SDNN(network_params, weight_params, stdp_params, total_time,
                     DoG_params=DoG_params, spike_times_learn=spike_times_learn,
                     spike_times_train=spike_times_train, spike_times_test=spike_times_test, device='GPU')


In [ ]:
# Set the weights or learn STDP
if set_weights:
    weight_path_list = ['/content/drive/MyDrive/Colab Notebooks/SDNN_ipynb/results'  + 'weight_' + str(i) + '.npy' for i in range(len(network_params) - 1)]
    first_net.set_weights(weight_path_list)
else:
    first_net.train_SDNN()


-----------------------------------------------------------
-------------------- STARTING LEARNING---------------------
-----------------------------------------------------------
----------------- Learning Progress  0/6500 (0.0)%----------------------
----------------- Learning Progress  1/6500 (0.015384615384615385)%----------------------
----------------- Learning Progress  2/6500 (0.03076923076923077)%----------------------

/content/SDNN_python/src/DoG_filt_cuda.py:64: RuntimeWarning: divide by zero encountered in true_divide
  I = np.argsort(1 / img_out.flatten())  # Get indices of sorted latencies
/content/SDNN_python/src/DoG_filt_cuda.py:65: RuntimeWarning: divide by zero encountered in true_divide
  lat = np.sort(1 / img_out.flatten())  # Get sorted latencies
/usr/local/lib/python3.9/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 16 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.9/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))



----------------- Learning Progress  3/6500 (0.046153846153846156)%----------------------
----------------- Learning Progress  4/6500 (0.06153846153846154)%----------------------
----------------- Learning Progress  5/6500 (0.07692307692307693)%----------------------
----------------- Learning Progress  6/6500 (0.09230769230769231)%----------------------
----------------- Learning Progress  7/6500 (0.1076923076923077)%----------------------
----------------- Learning Progress  8/6500 (0.12307692307692308)%----------------------
----------------- Learning Progress  9/6500 (0.13846153846153847)%----------------------
----------------- Learning Progress  10/6500 (0.15384615384615385)%----------------------
----------------- Learning Progress  11/6500 (0.16923076923076924)%----------------------
----------------- Learning Progress  12/6500 (0.18461538461538463)%----------------------
----------------- Learning Progress  13/6500 (0.2)%----------------------
----------------- Learning Progr

/usr/local/lib/python3.9/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.9/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Streaming output truncated to the last 5000 lines.
----------------- Learning Progress  1504/6500 (23.138461538461538)%----------------------
----------------- Learning Progress  1505/6500 (23.153846153846153)%----------------------
----------------- Learning Progress  1506/6500 (23.16923076923077)%----------------------
----------------- Learning Progress  1507/6500 (23.184615384615384)%----------------------
----------------- Learning Progress  1508/6500 (23.2)%----------------------
----------------- Learning Progress  1509/6500 (23.215384615384615)%----------------------
----------------- Learning Progress  1510/6500 (23.23076923076923)%----------------------
----------------- Learning Progress  1511/6500 (23.246153846153845)%----------------------
----------------- Learning Progress  1512/6500 (23.26153846153846)%----------------------
----------------- Learning Progress  1513/6500 (23.276923076923076)%----------------------
----------------- Learning Progress  1514/6500 (23.29230

In [ ]:
# Save the weights
ipynb_path2 = '/content/SDNN_python/results'
if save_weights:
    weights = first_net.get_weights()
    for i in range(len(weights)):
        pathsvwei = ipynb_path2 +'/weight_'+f"{i}"
        np.save(pathsvwei, weights[i])


In [ ]:
# Get features
X_train, y_train = first_net.train_features()
X_test, y_test = first_net.test_features()


Streaming output truncated to the last 5000 lines.
------------ Test features Extraction Progress  4709/10000 (47.09)%----------------
------------ Test features Extraction Progress  4710/10000 (47.1)%----------------
------------ Test features Extraction Progress  4711/10000 (47.11)%----------------
------------ Test features Extraction Progress  4712/10000 (47.12)%----------------
------------ Test features Extraction Progress  4713/10000 (47.13)%----------------
------------ Test features Extraction Progress  4714/10000 (47.14)%----------------
------------ Test features Extraction Progress  4715/10000 (47.15)%----------------
------------ Test features Extraction Progress  4716/10000 (47.16)%----------------
------------ Test features Extraction Progress  4717/10000 (47.17)%----------------
------------ Test features Extraction Progress  4718/10000 (47.18)%----------------
------------ Test features Extraction Progress  4719/10000 (47.19)%----------------
------------ Test features

In [25]:
# Save X_train and X_test
if save_features:
    np.save(ipynb_path + '/X_train', X_train)
    np.save(ipynb_path + '/y_train', y_train)
    np.save(ipynb_path + '/X_test', X_test)
    np.save(ipynb_path + '/y_test', y_test)


In [26]:
# ------------------------------- Classify -------------------------------#
classifier_params = {'C': 1.0, 'gamma': 'auto'}
train_mean = np.mean(X_train, axis=0)
train_std = np.std(X_train, axis=0)
X_train -= train_mean
X_test -= train_mean
X_train /= (train_std + 1e-5)
X_test /= (train_std + 1e-5)
svm = Classifier(X_train, y_train, X_test, y_test, classifier_params, classifier_type='SVM')
train_score, test_score = svm.run_classiffier()
print('Train Score: ' + str(train_score))
print('Test Score: ' + str(test_score))
print('DONE')

end = time.time()
print(end-start)

kernel was not specified
probability_flag was not specified


ValueError: ignored